In [1]:
from __future__ import print_function
import numpy as np # para computação numética menos intensiva
import os # para criar pastas
from matplotlib import pyplot as plt # para mostrar imagens
import tensorflow as tf # para redes neurais
from keras.datasets import cifar10
import keras.utils
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

c:\users\jordy\anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# criamos uma pasta para salvar o modelo
if not os.path.exists('tmp'): # se a pasta não existir
    os.makedirs('tmp')        # cria a pasta

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

num_train, img_rows, img_cols, img_channels =  x_train.shape

num_test, _, _, _ =  x_test.shape
num_classes = len(np.unique(y_train))

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [4]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range=0.3,
    zoom_range=0.3,
    fill_mode='nearest')
datagen.fit(x_train)

In [5]:
# definindo constantes
lr = 0.0001       # taxa de aprendizado
epochs = 500      # número de iterações de treino
batch_size = 500  # qtd de imagens no mini-lote 

logs_path = '/tmp/tensorflow_logs/example/' # caminho do log para o tensorboard

In [6]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():   
    x = tf.placeholder(tf.float32, shape=[None, img_rows, img_cols, img_channels], name='x')
    y = tf.placeholder(tf.float32, shape= [None,num_classes], name='y_input')
    
    input_layer = tf.reshape(x, [-1, img_rows, img_cols, img_channels])
    # Convolutional Layer 1
    conv1 = tf.layers.conv2d(inputs=x,
                             filters=32,
                             kernel_size=[8,8],
                             padding="same",
                             activation=tf.nn.selu,
                             name="conv1")

    # Pooling Layer 1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, 
                                    pool_size=[2,2], 
                                    strides=2,
                                    name="pool1")

    # Convolutional Layer 2
    conv2 = tf.layers.conv2d(inputs=pool1,
                             filters=32,
                             kernel_size=[16, 16],
                             padding="same",
                             activation=tf.nn.selu,
                             name="conv2")
        
    
    # Pooling Layer 2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, 
                                    pool_size=[2,2], 
                                    strides=2,
                                    name="pool2")
    
    # Convolutional Layer 3
    conv3 = tf.layers.conv2d(inputs=pool2,
                             filters=64,
                             kernel_size=[32, 32],
                             padding="same",
                             activation=tf.nn.selu,
                             name="conv3")
    # Pooling Layer 3
    pool3 = tf.layers.max_pooling2d(inputs=conv3, 
                                    pool_size=[2, 2], 
                                    strides=2,
                                    name="pool2")
    
    
     # Convolutional Layer 4
    conv4 = tf.layers.conv2d(inputs=pool3,
                             filters=16,
                             kernel_size=[8, 8],
                             padding="same",
                             activation=tf.nn.selu,
                             name="conv4")
    
    # Pooling Layer 5
    pool4 = tf.layers.max_pooling2d(inputs=conv4, 
                                    pool_size=[2, 2], 
                                    strides=2,
                                    name="pool4")
     
    # Convolutional Layer 5
    conv5 = tf.layers.conv2d(inputs=pool4,
                             filters=32,
                             kernel_size=[8, 8],
                             padding="same",
                             activation=tf.nn.selu,
                             name="conv5")
    # Pooling Layer 5
    pool5 = tf.layers.max_pooling2d(inputs=conv5, 
                                    pool_size=[2, 2], 
                                    strides=2,
                                    name="pool5")

    # Dense Layer
    flat = tf.contrib.layers.flatten(pool5) 
    dense = tf.layers.dense(inputs=flat, 
                            units=1024, 
                            activation=tf.nn.relu)
    
    # Dropout
    dropout = tf.layers.dropout(inputs=dense, 
                                rate=0.4)

    # Logits Layer
    logits = tf.layers.dense(inputs = dropout,
                             units  = num_classes)

    # Error Layer
    with tf.name_scope('Error_layer'):
        error = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits),
                               name='error')
        tf.summary.scalar('Cross_entropy', error) # para registrar a função custo

    # Accuracy
    with tf.name_scope("Accuracy"):
        correct_pred = tf.equal(tf.argmax(logits, 1), tf.cast(y, tf.int64))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        tf.summary.scalar('Accuracy', accuracy) 

    # Optimizer
    with tf.name_scope('Train_operation'):
        optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(error)

    # Inicialyzer
    init = tf.global_variables_initializer()

    # Save model
    saver = tf.train.Saver()

    # Visualization
    summaries = tf.summary.merge_all() # funde todos os summaries em uma operação
    file_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph()) # para escrever arquivos summaries

In [7]:
# abrimos a sessão tf
with tf.Session(graph=graph) as sess:
    #init.run() # iniciamos as variáveis
    # Run the initializer
    sess.run(init)
    
    # loop de treinamento
    for step in range(epochs+1):
        avg_cost = 0.
        # Number of images in the training-set.
        num_images = len(y_train)

        # Create a random index.
        idx = np.random.choice(num_images,
                               size=batch_size,
                               replace=False)

        # Use the random index to select random images and labels.      
        x_batch = x_train[idx, :, :, :]
        y_batch = y_train[idx, :]

        # cria um feed_dict
        feed_dict = {x: x_batch, y: y_batch}
        c, _ = sess.run([error, optimizer], feed_dict=feed_dict)
        avg_cost += c / num_images
        
        print('Epoca:', '%04d' % (step + 1), 'perda =', '{:.9f}'.format(avg_cost))

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    score =  sess.run(accuracy, feed_dict={x: x_test, y: y_test}) * 100
    print("Taxa de acerto: % .3f%%" % score)
        
    # Calculate accuracy
    #print("Accuracy:", accuracy.eval({x: x_test, y: y_test}))

    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Epoca: 0001 perda = 0.000046846
Epoca: 0002 perda = 0.000045982
Epoca: 0003 perda = 0.000046001
Epoca: 0004 perda = 0.000045657
Epoca: 0005 perda = 0.000045483
Epoca: 0006 perda = 0.000045107
Epoca: 0007 perda = 0.000044900
Epoca: 0008 perda = 0.000044952
Epoca: 0009 perda = 0.000044402
Epoca: 0010 perda = 0.000043844
Epoca: 0011 perda = 0.000042515
Epoca: 0012 perda = 0.000043076
Epoca: 0013 perda = 0.000042369
Epoca: 0014 perda = 0.000041998
Epoca: 0015 perda = 0.000041888
Epoca: 0016 perda = 0.000040531
Epoca: 0017 perda = 0.000039987
Epoca: 0018 perda = 0.000040880
Epoca: 0019 perda = 0.000040226
Epoca: 0020 perda = 0.000038833
Epoca: 0021 perda = 0.000040613
Epoca: 0022 perda = 0.000040048
Epoca: 0023 perda = 0.000039510
Epoca: 0024 perda = 0.000037701
Epoca: 0025 perda = 0.000039856
Epoca: 0026 perda = 0.000037026
Epoca: 0027 perda = 0.000038066
Epoca: 0028 perda = 0.000037561
Epoca: 0029 perda = 0.000037238
Epoca: 0030 perda = 0.000038840
Epoca: 0031 perda = 0.000039044
Epoca: 0

Epoca: 0259 perda = 0.000027250
Epoca: 0260 perda = 0.000026987
Epoca: 0261 perda = 0.000026817
Epoca: 0262 perda = 0.000025493
Epoca: 0263 perda = 0.000026664
Epoca: 0264 perda = 0.000026411
Epoca: 0265 perda = 0.000029278
Epoca: 0266 perda = 0.000023411
Epoca: 0267 perda = 0.000025916
Epoca: 0268 perda = 0.000025551
Epoca: 0269 perda = 0.000027600
Epoca: 0270 perda = 0.000026835
Epoca: 0271 perda = 0.000027124
Epoca: 0272 perda = 0.000026793
Epoca: 0273 perda = 0.000026393
Epoca: 0274 perda = 0.000026728
Epoca: 0275 perda = 0.000024606
Epoca: 0276 perda = 0.000025985
Epoca: 0277 perda = 0.000026980
Epoca: 0278 perda = 0.000024539
Epoca: 0279 perda = 0.000024826
Epoca: 0280 perda = 0.000027166
Epoca: 0281 perda = 0.000025442
Epoca: 0282 perda = 0.000025123
Epoca: 0283 perda = 0.000026854
Epoca: 0284 perda = 0.000026887
Epoca: 0285 perda = 0.000026033
Epoca: 0286 perda = 0.000025532
Epoca: 0287 perda = 0.000025195
Epoca: 0288 perda = 0.000027135
Epoca: 0289 perda = 0.000024601
Epoca: 0

In [8]:
file_writer.close() # fechamos o nó de escrever no disco.